In [ ]:
%pylab inline
#!/usr/bin/env python -u
# -*- coding: utf-8 -*-

import os
import cv2
import glob
import imageio
import numpy as np

## Install the following Python libraries:
##   pip install cffi rotlib skylibs
##
## Tip:
## If you get an error like "OSError: cannot load library '...\ezexr\libstdc++-6.dll': error 0x7e.",
## comment out the line "from hdrio import imread" in ...\Lib\site-packages\envmap\environmentmap.py.
import envmap

In [ ]:
def assembleEquirectFromCubemap(cubemap_prefix):

    # top, bottom, left, right, front, back
    cube_sides = ['u', 'd', 'l', 'r', 'f', 'b']
    cube_sides = [cubemap_prefix + e + ".png" for e in cube_sides]
    cube_sides = [np.array(imageio.imread(e) / 255.) for e in cube_sides]

    env = envmap.EnvironmentMap.fromSkybox(*cube_sides)
    env.convertTo('latlong', targetDim=cube_sides[0].shape[0])  # 1K cubemap => 2K equirect
#     env.convertTo('latlong', targetDim=(3 * cube_sides[0].shape[0]) // 2)  # 1K cubemap => 3K equirect
    equirect = (255 * env.data).astype(np.uint8)

    imageio.imwrite(cubemap_prefix[:-8] + f"equirect-{equirect.shape[1]}.jpg", equirect, quality=90, optimize=True)
    imageio.imwrite(cubemap_prefix[:-8] + f"equirect-{equirect.shape[1]}.png", equirect)

In [ ]:
def assembleEquirectFromReplicaCubemap(cubemap_prefix):

    # top, bottom, left, right, front, back
    cube_sides = ['U', 'D', 'L', 'R', 'F', 'B']
    cube_sides = [cubemap_prefix + e + ".jpg" for e in cube_sides]
    cube_sides = [np.array(imageio.imread(e) / 255.) for e in cube_sides]
    cube_sides[0] = cv2.rotate(cube_sides[0], 0) ## rotate up view
    cube_sides[1] = cv2.rotate(cube_sides[1], 2) ## rotate down view

    env = envmap.EnvironmentMap.fromSkybox(*cube_sides)
#     env.convertTo('latlong', targetDim=cube_sides[0].shape[0])
    env.convertTo('latlong', targetDim=(3 * cube_sides[0].shape[0]) // 2)
    equirect = (255 * env.data).astype(np.uint8)

    imageio.imwrite(cubemap_prefix + f"equirect-{equirect.shape[1]}.jpg", equirect, quality=90, optimize=True)
#     imageio.imwrite(cubemap_prefix + f"equirect-{equirect.shape[1]}.png", equirect)

# assembleEquirectFromReplicaCubemap(r"G:\OmniPhoto\Data\GT-Replica\room_1\cubmap_image\image\room_1_0040_")

In [ ]:
code_dir = os.path.dirname(os.getcwd())
screenshot_dir = os.path.join(code_dir, 'Screenshots')
cubemaps = glob.glob(screenshot_dir + "\*-cubemap-f.png")

for f in cubemaps:
    cubemap_prefix_with_trailing_hyphen = f[:-5]
    print(f"Assembling cubemap '{cubemap_prefix_with_trailing_hyphen[:-9]}' ...")
    assembleEquirectFromCubemap(cubemap_prefix_with_trailing_hyphen)

In [ ]:
# ## Lets try ERP to cubemap
# filename = r"G:\OmniPhotos\Data\ParadeGardens\Input\panoramic-0186.jpg"
# image = np.array(imageio.imread(filename) / 255.)  # convert to unit range RGB
# # imshow(image)

# env = envmap.EnvironmentMap(image, 'latlong')
# env.convertTo('cube')
# imshow(env.data)